In [7]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd

%load_ext lab_black
%load_ext watermark

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Magnesium for myocardial infarction

This example demonstrates ...

Adapted from [unit 10: magnesiumTW.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/magnesiumTW.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

No associated lecture video?

## Problem statement

The randomized controlled trials of magnesium treatment in the prevention of death following myocardial infarction are a well known example where publication bias, demonstrated by an asymmetrical funnel plot, has been suggested
as an explanation for the discrepancy
between meta-analyses which showed a clear beneficial
effect of magnesium therapy on mortality (Teo et al, 1991; Horner 1992) and a
subsequent large trial which showed no effect (ISIS-4).

 
Results from 16 trials of intravenous magnesium in acute
myocardial infarction
                         Mg.           Control

Study                    Deaths Total  Deaths Total


- Morton  1984            1   40         2    36 
- Rasmussen  1986     9  135       23   135
- Smith  1986              2  200        7  200
- Abraham  1987         1   48         1  46
- Feldstedt  1988       10  150        8   148 
- Schechter  1989       1   59         9   56
- Ceremuzynski  1989  1  25         3  23
- Bertschat  1989          0  22       1  21
- Singh  &1990 & 6 &76 &11 &75\\
- Pereira &1990 &1 &27 &7 &27\\
- Schechter \& Hod 1 &1991 &2 &89 &12 &80\\
- Golf &1991 &5 &23 &13 &33\\
- Thogersen  1991     4  130    8  122 
- LIMIT-2 &1992 &90 &1159& 118 &1157\\
- Schechter 1995    4  107  17 &108 
- ISIS-4  1995  2216 29011  2103  29039 


(a) Using a Bayesian model, WinBUGS, and log odds ratio as a study effect,
conduct a Bayesian meta-analysis. Discuss your findings.


(b) Read about publication bias and funnel plots. Plot the posterior means for log odds ratios against posterior standard deviations. (A Bayesian version of funnel plot) Discuss your findings.


Teo, K. K., Yusuf, S., Collins, R., Held, P. H., and Peto, R. (1991). Effects of intravenous magnesium in suspected acute myocardial infarction: overview of randomised trials. {\it Br. Med. J.}, {\bf 303}, 1499--1503.

Horner S. M. (1992). Efficacy of intravenous magnesium in acute myocardial infarction in reducing arrhythmias and mortality: Meta-analysis of magnesium in acute myocardial infarction. {\it Circulation}, {\bf 86}, 774--779.

ISIS-4 Collaborative Group. (1995). ISIS-4: A randomised factorial trial assessing early oral captopril, oral mononitrate, and intravenous magnesium sulphate in 58,050 patients with suspected acute myocardial infarction. {\it Lancet}, {\bf 345}, 669--687.

In [3]:
data = pd.read_csv("../data/magnesiumtw.csv")
data

,D.Magn,N.Magn,D.Controls,N.Controls
0,1,40,2,36
1,9,135,23,135
2,2,200,7,200
3,1,48,1,46
4,10,150,8,148
5,1,59,9,56
6,1,25,3,23
7,0,22,1,21
8,6,76,11,75
9,1,27,7,27


In [4]:
d_magn = data["D.Magn"].to_numpy()
d_controls = data["D.Controls"].to_numpy()
n_magn = data["N.Magn"].to_numpy()
n_controls = data["N.Controls"].to_numpy()

In [8]:
with pm.Model() as m:
    T = pm.Gamma("T", 0.001, 0.001)
    mu = pm.Normal("mu", 0, tau=1e-5)
    # rename these mus, super confusing
    Mu = pm.Normal("Mu", 0, 1e-4)
    effect = pm.Normal("effect", Mu, tau=T)

    p_magn = mu + effect
    p_controls = mu

    pm.Binomial("d_controls", logit_p=p_controls, n=n_controls, observed=d_controls)
    pm.Binomial("d_magn", logit_p=p_magn, n=n_magn, observed=d_magn)

    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [T, mu, Mu, effect]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 15 seconds.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase 

In [9]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,-2.514,0.021,-2.552,-2.473,0.000,0.000,6106.0,6349.0,1.0
Mu,-0.000,0.000,-0.000,0.000,0.000,0.000,5898.0,6234.0,1.0
effect,0.003,0.028,-0.048,0.056,0.000,0.000,5880.0,6648.0,1.0
T,399.962,588.616,0.000,1433.828,6.557,4.942,4641.0,2723.0,1.0
